# 숙제 1번

In [1]:
import numpy as np

In [12]:
# zero padding을 크기 1만큼 적용 
a=np.array([[0, 0, 0, 0, 0, 0, 0, 0],
            [0, 2, 0, 2, 1, 3, 2, 0],
            [0, 0, 2, 0, 2, 2, 2, 0],
            [0, 1, 0, 1, 3, 1, 1, 0],
            [0, 0, 0, 1, 1, 1, 0, 0],
            [0, 0, 1, 3, 4, 1, 0, 0],
            [0, 0, 1, 0, 0, 5, 2, 0],
            [0, 0, 0, 0, 0, 0, 0, 0]])
a

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [36]:
A = np.array([[1, 0, 0],
               [0, 1, 0],
               [0, 0, 1]])
A

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [102]:
r = [0,1,2,3,4,5]
k = np.empty((6 ,6), int)
k

array([[4128860, 6029375, 3801155, 5570652, 6619251, 7536754],
       [7536732, 7143521, 7667827, 6750318, 3014748, 7340137],
       [7602297, 7274600, 6029422, 7471216, 6684783, 7077993],
       [6226021, 6619236, 6357094, 7078005, 6029428, 6881384],
       [7602291, 7471215, 3014777, 7405683, 6881388, 6619252],
       [6946861, 7667823, 7209074, 7077985, 3014656, 7929968]])

In [103]:
for j in r:
    for i in r:
        kk= a[j:j+3,i:i+3] * A
        k[j,i]= sum(sum(kk))

In [104]:
# stride (1,1)로 filter A 적용 
k

array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

In [98]:
k1 = np.empty((3,3), int)
rr = [0, 1, 2]

In [100]:
    for j in rr:
        for i in rr:
            k1[j, i] = np.max(k[j*2:j*2+2, i*2:i*2+2])
            

In [101]:
# stride(2,2)로 maxpooling 적용 
k1

array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

# 숙제 2번

In [115]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [116]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

In [126]:
# Parameters
learning_rate=0.001
training_epochs=10
batch_size=100

In [131]:
# MNIST dataset

mnist_train = dsets.MNIST(root="MNIST_data/",
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root="MNIST_data/",
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw
Processing...


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


In [203]:
# Data Loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

우선, Layer1의 Kernel_size는 4로, Layer3의 stride는 2로 조정하고<br>
그 결과 달라지는 Layer3 의 output size계산 <br>
$Output\; size = \frac{input \;size-filter \;size+(2*padding)}{Stride}+1$

In [201]:
layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )        

In [202]:
out = layer1(X) 
out = layer2(out)
out = layer3(out)
out.shape

torch.Size([100, 128, 2, 2])

In [210]:
# Model2

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )        
      

        self.fc1 = nn.Linear(2*2*128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight) 
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p=1-self.keep_prob))
        
        
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight) 
        
    
    def forward(self, x):
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # fc돌리기 전에 view로 펼치기 
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [211]:
model = CNN().to(device)  # device는 cuda, cpu 중에서 사용가능한거 쓰겟다 옵션 

In [212]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [214]:
# Training 
total_batch = len(data_loader)
print(total_batch)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader : 
        X=X.to(device) # 이미지 (input)
        Y=Y.to(device) # 라벨
        
        optimizer.zero_grad() ## 중요중요 ##
        hypothesis = model(X) # model에 X input 대입한 결과 (가설)
        
        cost = criterion(hypothesis, Y) # 그 가설과 실제 라벨 사이의 차이 계산(Loss)
        cost.backward()
        optimizer.step() # 모델 학습 
        
        avg_cost += cost / total_batch # cost 쌓기 
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')

600
[Epoch:1] cost = 0.10703402012586594
[Epoch:2] cost = 0.056733112782239914
[Epoch:3] cost = 0.04211655631661415
[Epoch:4] cost = 0.03434012085199356
[Epoch:5] cost = 0.025526992976665497
[Epoch:6] cost = 0.023407865315675735
[Epoch:7] cost = 0.019292963668704033
[Epoch:8] cost = 0.01732758618891239
[Epoch:9] cost = 0.01442783698439598
[Epoch:10] cost = 0.014809113927185535
Learning Finished!


In [215]:
with torch.no_grad(): # test는 학습을 안할꺼니까 no gradient
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accurary:', accuracy.item())

Accurary: 0.9776999950408936


# 기존 CNN모델 

In [216]:
# Model2

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )        
      

        self.fc1 = nn.Linear(4*4*128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight) 
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p=1-self.keep_prob))
        
        
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight) 
        
    
    def forward(self, x):
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # fc돌리기 전에 view로 펼치기 
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [217]:
model = CNN().to(device)

In [218]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [219]:
# Training 
total_batch = len(data_loader)
print(total_batch)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader : 
        X=X.to(device) # 이미지 (input)
        Y=Y.to(device) # 라벨
        
        optimizer.zero_grad() ## 중요중요 ##
        hypothesis = model(X) # model에 X input 대입한 결과 (가설)
        
        cost = criterion(hypothesis, Y) # 그 가설과 실제 라벨 사이의 차이 계산(Loss)
        cost.backward()
        optimizer.step() # 모델 학습 
        
        avg_cost += cost / total_batch # cost 쌓기 
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')

600
[Epoch:1] cost = 0.18747356534004211
[Epoch:2] cost = 0.054013680666685104
[Epoch:3] cost = 0.03851877152919769
[Epoch:4] cost = 0.02870580367743969
[Epoch:5] cost = 0.02470463141798973
[Epoch:6] cost = 0.020793743431568146
[Epoch:7] cost = 0.01785752736032009
[Epoch:8] cost = 0.01366757694631815
[Epoch:9] cost = 0.011903111822903156
[Epoch:10] cost = 0.012358635663986206
Learning Finished!


In [220]:
with torch.no_grad(): # test는 학습을 안할꺼니까 no gradient
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accurary:', accuracy.item())

Accurary: 0.9905999898910522
